In [1]:
import sys
import os
import json
import random

from PyQt5 import QtGui  # for QFont
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel,
    QMessageBox, QComboBox, QStackedWidget, QPushButton, QRadioButton,
    QButtonGroup
)
from PyQt5.QtCore import Qt

JSON_FILE = "languages.json"

def load_data():
    """Load JSON data from the languages.json file."""
    if not os.path.exists(JSON_FILE):
        print(f"[DEBUG] JSON file '{JSON_FILE}' does not exist.")
        return {}
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            print("[DEBUG] Successfully loaded data:", data)  # Debug print
            return data
        except json.JSONDecodeError as e:
            print("[DEBUG] JSONDecodeError:", e)
            return {}

class LanguageSelectionPage(QWidget):
    def __init__(self, stacked_widget):
        super().__init__()
        self.stacked_widget = stacked_widget
        self.languages_data = load_data()
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()
        label = QLabel("Select a language:")
        label.setStyleSheet("font-size: 16px;")  # bigger label
        layout.addWidget(label)

        self.language_combo = QComboBox()
        self.language_combo.addItems(self.languages_data.keys())
        layout.addWidget(self.language_combo)

        self.continue_button = QPushButton("Continue")
        self.continue_button.setStyleSheet("font-size: 16px;")
        self.continue_button.clicked.connect(self.continue_to_morph_page)
        layout.addWidget(self.continue_button)

        self.setLayout(layout)

    def continue_to_morph_page(self):
        selected_lang = self.language_combo.currentText().strip()
        if not selected_lang:
            QMessageBox.warning(self, "Error", "Please select a language.")
            return

        # Go to the second page (MorphSplitPage) and set its current language
        morph_page = self.stacked_widget.widget(1)
        morph_page.set_current_language(selected_lang, self.languages_data)
        self.stacked_widget.setCurrentIndex(1)

class MorphSplitPage(QWidget):
    def __init__(self, stacked_widget):
        super().__init__()
        self.stacked_widget = stacked_widget  # Store reference to switch pages
        self.current_language = None
        self.vocab_dict = {}
        self.current_word = None
        self.parts_list = []
        self.current_part_index = 0
        self.initUI()

    def initUI(self):
        self.setWindowTitle("Morphological Splitting Tester (Interactive)")
        main_layout = QVBoxLayout(self)
        self.setLayout(main_layout)

        self.prompt_label = QLabel("Click 'Test Me' to start.")
        self.prompt_label.setStyleSheet("font-size: 16px;")
        main_layout.addWidget(self.prompt_label, alignment=Qt.AlignCenter)

        btn_layout = QHBoxLayout()
        main_layout.addLayout(btn_layout)

        # -- ADD BACK BUTTON --
        self.back_button = QPushButton("Back")
        self.back_button.setStyleSheet("font-size: 16px;")
        self.back_button.clicked.connect(self.go_back)
        btn_layout.addWidget(self.back_button)

        self.test_me_button = QPushButton("Test Me")
        self.test_me_button.setStyleSheet("font-size: 16px;")
        self.test_me_button.clicked.connect(self.start_test)
        btn_layout.addWidget(self.test_me_button)

        # Container for question + radio buttons
        self.question_container = QWidget()
        self.question_layout = QVBoxLayout(self.question_container)
        main_layout.addWidget(self.question_container)

        self.question_label = QLabel("")
        self.question_label.setStyleSheet("font-weight: bold; font-size: 16px;")
        self.question_layout.addWidget(self.question_label)

        # Radio buttons
        self.button_group = QButtonGroup(self)
        self.radio_buttons = []
        for i in range(5):
            rb = QRadioButton()
            # No forced color, just bigger font
            rb.setStyleSheet("font-size: 16px;")
            self.radio_buttons.append(rb)
            self.button_group.addButton(rb)
            self.question_layout.addWidget(rb)

        # Action buttons for checking and going to next
        action_btns_layout = QHBoxLayout()
        self.question_layout.addLayout(action_btns_layout)

        self.check_button = QPushButton("Check Answer")
        self.check_button.setStyleSheet("font-size: 16px;")
        self.check_button.clicked.connect(self.check_answer)
        action_btns_layout.addWidget(self.check_button)

        self.next_button = QPushButton("Next")
        self.next_button.setStyleSheet("font-size: 16px;")
        self.next_button.clicked.connect(self.next_part)
        self.next_button.setEnabled(False)
        action_btns_layout.addWidget(self.next_button)

        self.hide_test_ui()  # Hide radio buttons area on startup

    def go_back(self):
        """Go back to the first (language selection) page."""
        self.stacked_widget.setCurrentIndex(0)

    def set_current_language(self, language, all_data):
        """Sets the current language data from the loaded JSON."""
        self.current_language = language
        self.setWindowTitle(f"Morphological Tester (Interactive) - {language}")
        self.vocab_dict = all_data.get(language, {})
        print(f"[DEBUG] Current language set to: {self.current_language}")
        print(f"[DEBUG] Loaded vocabulary dict: {self.vocab_dict}")

    def start_test(self):
        """Pick a random word and start testing."""
        if not self.vocab_dict:
            QMessageBox.warning(self, "No Words Found", "No words found in this language.")
            return

        self.current_word = random.choice(list(self.vocab_dict.keys()))
        self.parts_list = self.vocab_dict[self.current_word].get("parts", [])
        self.current_part_index = 0

        print(f"[DEBUG] Selected word: {self.current_word}")
        print(f"[DEBUG] Parts list: {self.parts_list}")

        if not self.parts_list:
            final_exp = self.vocab_dict[self.current_word].get("final_explanation", "")
            QMessageBox.information(self, "No Parts", final_exp or "No splitting info.")
            return

        self.prompt_label.setText(f"Testing word: {self.current_word}")
        self.show_test_ui()
        self.show_part_question()

    def show_part_question(self):
        """Displays the question for the current part."""
        if self.current_part_index >= len(self.parts_list):
            self.show_final_explanation()
            return

        # Reset radio buttons
        self.button_group.setExclusive(False)
        for rb in self.radio_buttons:
            rb.setChecked(False)
            rb.hide()
        self.button_group.setExclusive(True)
        self.reset_radio_styles()

        # Re-enable "Check Answer" and disable "Next" until user checks
        self.check_button.setEnabled(True)
        self.next_button.setEnabled(False)

        part_data = self.parts_list[self.current_part_index]
        part_label = part_data["label"]
        options = part_data["options"]
        correct_answer = part_data["correct"]

        # If the correct answer isn't in the options, prepend it
        if correct_answer not in options:
            options = [correct_answer] + options

        # Shuffle the options so the correct answer isn't always first
        random_options = list(set(options))  # remove duplicates
        random.shuffle(random_options)

        self.question_label.setText(
            f"Which {part_label} is correct for \"{self.current_word}\"?"
        )

        print(f"[DEBUG] Showing question for part: {part_label}")
        print(f"[DEBUG] Randomized options: {random_options}")

        # Show as many radio buttons as there are options (up to 5)
        for i, opt in enumerate(random_options):
            if i < len(self.radio_buttons):
                self.radio_buttons[i].setText(opt)
                self.radio_buttons[i].show()

    def check_answer(self):
        """Check the user's selected answer against the correct one."""
        if self.current_part_index >= len(self.parts_list):
            return

        selected_rb = self.button_group.checkedButton()
        if not selected_rb:
            QMessageBox.warning(self, "No Selection", "Please select an option.")
            return

        user_choice = selected_rb.text()
        part_data = self.parts_list[self.current_part_index]
        correct_answer = part_data["correct"]
        explanation = part_data["explanation"]

        # Once the user checks, disable the check button, enable next
        self.check_button.setEnabled(False)
        self.next_button.setEnabled(True)

        if user_choice == correct_answer:
            # Mark the selected radio button in green
            selected_rb.setStyleSheet("font-size: 16px; color: green; font-weight: bold;")
            QMessageBox.information(
                self,
                "Correct!",
                f"You chose '{user_choice}'.\n\nExplanation:\n{explanation}"
            )
        else:
            # Mark the selected (wrong) radio button in red
            selected_rb.setStyleSheet("font-size: 16px; color: red; font-weight: bold;")
            # Highlight the correct one in green
            for rb in self.radio_buttons:
                if rb.text() == correct_answer:
                    rb.setStyleSheet("font-size: 16px; color: green; font-weight: bold;")
                    break
            QMessageBox.warning(
                self,
                "Incorrect",
                f"You chose '{user_choice}'. The correct is '{correct_answer}'.\n\nExplanation:\n{explanation}"
            )

    def next_part(self):
        """Go to the next part (prefix/root/suffix/etc.) if any."""
        self.current_part_index += 1
        if self.current_part_index < len(self.parts_list):
            self.show_part_question()
        else:
            self.show_final_explanation()

    def show_final_explanation(self):
        """Show the final explanation for the current word."""
        final_exp = self.vocab_dict[self.current_word].get("final_explanation", "")
        if final_exp:
            QMessageBox.information(self, "Splitting Explanation", final_exp)
        self.hide_test_ui()

    def hide_test_ui(self):
        """Hide the question UI, used at startup and after finishing a word."""
        self.question_label.hide()
        for rb in self.radio_buttons:
            rb.hide()
        self.check_button.hide()
        self.next_button.hide()

    def show_test_ui(self):
        """Show the question UI elements."""
        self.question_label.show()
        self.check_button.show()
        self.next_button.setEnabled(False)
        self.next_button.show()

    def reset_radio_styles(self):
        """Reset radio buttons to a neutral style (default system color)."""
        for rb in self.radio_buttons:
            rb.setStyleSheet("font-size: 16px;")

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Morphological Splitting App (Interactive)")
        self.stacked_widget = QStackedWidget()

        # First page (language selection)
        self.language_selection_page = LanguageSelectionPage(self.stacked_widget)

        # Second page (morph split testing) -- pass stacked_widget so we can go back
        self.morph_page = MorphSplitPage(self.stacked_widget)

        self.stacked_widget.addWidget(self.language_selection_page)
        self.stacked_widget.addWidget(self.morph_page)

        layout = QVBoxLayout()
        layout.addWidget(self.stacked_widget)
        self.setLayout(layout)
        # Show the first page by default
        self.stacked_widget.setCurrentIndex(0)

def main():
    app = QApplication(sys.argv)

    # 1) Set a bigger global font for the entire application
    font = QtGui.QFont()
    font.setPointSize(20)  # Increase default size
    app.setFont(font)

    window = MainWindow()
    # 2) Start at a larger size; the user can still resize or maximize
    window.resize(900, 600)
    window.show()

    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


[DEBUG] Successfully loaded data: {'Russian': {'отличается': {'parts': [{'label': 'prefix', 'correct': 'от', 'options': ['от', 'о', 'отли'], 'explanation': "от- means 'from' or 'away from'"}, {'label': 'root', 'correct': 'личается', 'options': ['личается', 'личаться', 'ликать', 'личать'], 'explanation': 'from ликовать (to exult, rejoice)'}], 'final_explanation': 'Splitting: от- (from) + личается (from ликовать, to exult, rejoice)'}, 'находит': {'parts': [{'label': 'prefix', 'correct': 'на', 'options': ['на', 'над', 'нах'], 'explanation': "на- can mean 'on' or 'onto'"}, {'label': 'root', 'correct': 'ходит', 'options': ['ходит', 'ходить', 'хоодит'], 'explanation': 'from ходить (to walk)'}], 'final_explanation': 'Splitting: на- + ходит (from ходить, to walk)'}}}


2025-02-07 14:02:16.601 python[12359:566569] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-07 14:02:16.601 python[12359:566569] +[IMKInputSession subclass]: chose IMKInputSession_Modern


[DEBUG] Current language set to: Russian
[DEBUG] Loaded vocabulary dict: {'отличается': {'parts': [{'label': 'prefix', 'correct': 'от', 'options': ['от', 'о', 'отли'], 'explanation': "от- means 'from' or 'away from'"}, {'label': 'root', 'correct': 'личается', 'options': ['личается', 'личаться', 'ликать', 'личать'], 'explanation': 'from ликовать (to exult, rejoice)'}], 'final_explanation': 'Splitting: от- (from) + личается (from ликовать, to exult, rejoice)'}, 'находит': {'parts': [{'label': 'prefix', 'correct': 'на', 'options': ['на', 'над', 'нах'], 'explanation': "на- can mean 'on' or 'onto'"}, {'label': 'root', 'correct': 'ходит', 'options': ['ходит', 'ходить', 'хоодит'], 'explanation': 'from ходить (to walk)'}], 'final_explanation': 'Splitting: на- + ходит (from ходить, to walk)'}}
[DEBUG] Selected word: отличается
[DEBUG] Parts list: [{'label': 'prefix', 'correct': 'от', 'options': ['от', 'о', 'отли'], 'explanation': "от- means 'from' or 'away from'"}, {'label': 'root', 'correct': 

SystemExit: 0

/Users/bladeruuner/miniforge3/envs/QNLP/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
import os
import json
import random
import ollama

from PyQt5 import QtGui  # for QFont
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel,
    QMessageBox, QComboBox, QStackedWidget, QPushButton, QRadioButton,
    QButtonGroup
)
from PyQt5.QtCore import Qt

JSON_FILE = "languages.json"

def load_data():
    """Load JSON data from the languages.json file."""
    if not os.path.exists(JSON_FILE):
        print(f"[DEBUG] JSON file '{JSON_FILE}' does not exist.")
        return {}
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            print("[DEBUG] Successfully loaded data:", data)
            return data
        except json.JSONDecodeError as e:
            print("[DEBUG] JSONDecodeError:", e)
            return {}

class LanguageSelectionPage(QWidget):
    def __init__(self, stacked_widget):
        super().__init__()
        self.stacked_widget = stacked_widget
        self.languages_data = load_data()
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()
        label = QLabel("Select a language:")
        label.setStyleSheet("font-size: 16px;")
        layout.addWidget(label)

        self.language_combo = QComboBox()
        self.language_combo.addItems(self.languages_data.keys())
        layout.addWidget(self.language_combo)

        self.continue_button = QPushButton("Continue")
        self.continue_button.setStyleSheet("font-size: 16px;")
        self.continue_button.clicked.connect(self.continue_to_morph_page)
        layout.addWidget(self.continue_button)

        self.setLayout(layout)

    def continue_to_morph_page(self):
        selected_lang = self.language_combo.currentText().strip()
        if not selected_lang:
            QMessageBox.warning(self, "Error", "Please select a language.")
            return

        morph_page = self.stacked_widget.widget(1)
        morph_page.set_current_language(selected_lang, self.languages_data)
        self.stacked_widget.setCurrentIndex(1)

class MorphSplitPage(QWidget):
    def __init__(self, stacked_widget):
        super().__init__()
        self.stacked_widget = stacked_widget
        self.current_language = None
        self.vocab_dict = {}
        self.current_word = None  # MODIFIED: track the current word after Ollama generation
        self.parts_list = []
        self.current_part_index = 0
        self.initUI()

    def initUI(self):
        self.setWindowTitle("Morphological Splitting Tester (Interactive)")
        main_layout = QVBoxLayout(self)
        self.setLayout(main_layout)

        self.prompt_label = QLabel("Click 'Test Me' to start.")
        self.prompt_label.setStyleSheet("font-size: 16px;")
        main_layout.addWidget(self.prompt_label, alignment=Qt.AlignCenter)

        # Row of buttons: Back, Test Me, Generate Ollama Question
        btn_layout = QHBoxLayout()
        main_layout.addLayout(btn_layout)

        self.back_button = QPushButton("Back")
        self.back_button.setStyleSheet("font-size: 16px;")
        self.back_button.clicked.connect(self.go_back)
        btn_layout.addWidget(self.back_button)

        self.test_me_button = QPushButton("Test Me")
        self.test_me_button.setStyleSheet("font-size: 16px;")
        self.test_me_button.clicked.connect(self.start_test)
        btn_layout.addWidget(self.test_me_button)

        # MODIFIED: Button to generate a memorization question with Ollama first
        self.ollama_button = QPushButton("Generate Ollama Question")
        self.ollama_button.setStyleSheet("font-size: 16px;")
        self.ollama_button.clicked.connect(self.generate_question)
        btn_layout.addWidget(self.ollama_button)

        self.question_container = QWidget()
        self.question_layout = QVBoxLayout(self.question_container)
        main_layout.addWidget(self.question_container)

        self.question_label = QLabel("")
        self.question_label.setStyleSheet("font-weight: bold; font-size: 16px;")
        self.question_layout.addWidget(self.question_label)

        self.button_group = QButtonGroup(self)
        self.radio_buttons = []
        for i in range(5):
            rb = QRadioButton()
            rb.setStyleSheet("font-size: 16px;")
            self.radio_buttons.append(rb)
            self.button_group.addButton(rb)
            self.question_layout.addWidget(rb)

        action_btns_layout = QHBoxLayout()
        self.question_layout.addLayout(action_btns_layout)

        self.check_button = QPushButton("Check Answer")
        self.check_button.setStyleSheet("font-size: 16px;")
        self.check_button.clicked.connect(self.check_answer)
        action_btns_layout.addWidget(self.check_button)

        self.next_button = QPushButton("Next")
        self.next_button.setStyleSheet("font-size: 16px;")
        self.next_button.clicked.connect(self.next_part)
        self.next_button.setEnabled(False)
        action_btns_layout.addWidget(self.next_button)

        self.hide_test_ui()

    def go_back(self):
        self.stacked_widget.setCurrentIndex(0)

    def set_current_language(self, language, all_data):
        self.current_language = language

        # If the JSON for this language is a list, convert to dict
        lang_data = all_data.get(language, {})
        if isinstance(lang_data, list):
            temp_dict = {}
            for w in lang_data:
                temp_dict[w] = {
                    "parts": [],
                    "final_explanation": f"No morphological splits found for '{w}'."
                }
            self.vocab_dict = temp_dict
        else:
            self.vocab_dict = lang_data

        self.setWindowTitle(f"Morphological Tester (Interactive) - {language}")

    def start_test(self):
        # MODIFIED: If the user has already generated an Ollama question, 
        #           self.current_word is set. Otherwise, choose a random one.
        if not self.vocab_dict:
            QMessageBox.warning(self, "No Words Found", "No words found in this language.")
            return

        if not self.current_word:
            # if user hasn't generated an Ollama question for a specific word
            self.current_word = random.choice(list(self.vocab_dict.keys()))

        self.parts_list = self.vocab_dict[self.current_word].get("parts", [])
        self.current_part_index = 0

        if not self.parts_list:
            final_exp = self.vocab_dict[self.current_word].get("final_explanation", "")
            QMessageBox.information(self, "No Parts", final_exp or "No splitting info.")
            # Reset self.current_word so we can pick another next time
            self.current_word = None
            return

        self.prompt_label.setText(f"Testing word: {self.current_word}")
        self.show_test_ui()
        self.show_part_question()

    def show_part_question(self):
        if self.current_part_index >= len(self.parts_list):
            self.show_final_explanation()
            return

        self.button_group.setExclusive(False)
        for rb in self.radio_buttons:
            rb.setChecked(False)
            rb.hide()
        self.button_group.setExclusive(True)
        self.reset_radio_styles()

        self.check_button.setEnabled(True)
        self.next_button.setEnabled(False)

        part_data = self.parts_list[self.current_part_index]
        part_label = part_data["label"]
        options = part_data["options"]
        correct_answer = part_data["correct"]

        # Make sure the correct answer is in the options
        if correct_answer not in options:
            options = [correct_answer] + options

        random_options = list(set(options))
        random.shuffle(random_options)

        self.question_label.setText(
            f"Which {part_label} is correct for \"{self.current_word}\"?"
        )

        for i, opt in enumerate(random_options):
            if i < len(self.radio_buttons):
                self.radio_buttons[i].setText(opt)
                self.radio_buttons[i].show()

    def check_answer(self):
        if self.current_part_index >= len(self.parts_list):
            return

        selected_rb = self.button_group.checkedButton()
        if not selected_rb:
            QMessageBox.warning(self, "No Selection", "Please select an option.")
            return

        user_choice = selected_rb.text()
        part_data = self.parts_list[self.current_part_index]
        correct_answer = part_data["correct"]
        explanation = part_data["explanation"]

        self.check_button.setEnabled(False)
        self.next_button.setEnabled(True)

        if user_choice == correct_answer:
            selected_rb.setStyleSheet("font-size: 16px; color: green; font-weight: bold;")
            QMessageBox.information(
                self, "Correct!",
                f"You chose '{user_choice}'.\n\nExplanation:\n{explanation}"
            )
        else:
            selected_rb.setStyleSheet("font-size: 16px; color: red; font-weight: bold;")
            for rb in self.radio_buttons:
                if rb.text() == correct_answer:
                    rb.setStyleSheet("font-size: 16px; color: green; font-weight: bold;")
                    break
            QMessageBox.warning(
                self, "Incorrect",
                f"You chose '{user_choice}'. The correct is '{correct_answer}'.\n\nExplanation:\n{explanation}"
            )

    def next_part(self):
        self.current_part_index += 1
        if self.current_part_index < len(self.parts_list):
            self.show_part_question()
        else:
            self.show_final_explanation()

    def show_final_explanation(self):
        final_exp = self.vocab_dict[self.current_word].get("final_explanation", "")
        if final_exp:
            QMessageBox.information(self, "Splitting Explanation", final_exp)
        # After finishing, reset current_word so if user clicks “Test Me” again 
        # it either picks a new random word or has to re-generate from Ollama.
        self.current_word = None
        self.hide_test_ui()

    def hide_test_ui(self):
        self.question_label.hide()
        for rb in self.radio_buttons:
            rb.hide()
        self.check_button.hide()
        self.next_button.hide()

    def show_test_ui(self):
        self.question_label.show()
        self.check_button.show()
        self.next_button.setEnabled(False)
        self.next_button.show()

    def reset_radio_styles(self):
        for rb in self.radio_buttons:
            rb.setStyleSheet("font-size: 16px;")

    # ------------------------------------------------------------------
    # MODIFIED: Generate a memorization prompt for a random (or specific) word
    #           and store that word in self.current_word, so the user
    #           can test themselves right after.
    # ------------------------------------------------------------------
    def generate_question(self):
        # 1) Make sure we have some words
        if not self.vocab_dict:
            QMessageBox.warning(self, "No Words Found", "No words found in this language.")
            return

        # 2) Pick a random word from the current language
        self.current_word = random.choice(list(self.vocab_dict.keys()))
        
        # 3) Build a prompt asking for memorization tips
        questionToAsk = (
            f"I want to memorize '{self.current_word}' in {self.current_language}. "
            f"Please provide a short mnemonic or memory technique, and example uses."
        )

        # 4) Call Ollama with your chosen model
        desiredModel = 'deepseek-r1:14b'
        response = ollama.chat(
            model=desiredModel,
            messages=[{'role': 'user', 'content': questionToAsk}]
        )

        # 5) Extract Ollama's text
        OllamaResponse = response['message']['content']

        # 6) (Optional) Write the response to a file
        with open("OutputOllama.txt", "w", encoding="utf-8") as text_file:
            text_file.write(OllamaResponse)

        # 7) Display in a popup
        QMessageBox.information(
            self,
            "Ollama Memorization Tip",
            f"**Word**: {self.current_word}\n\n**Ollama's Suggestion**:\n{OllamaResponse}"
        )

        # Now self.current_word holds the random word from Ollama, so if the user
        # clicks "Test Me", it will quiz them on this same word immediately.

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Morphological Splitting App (Interactive)")
        self.stacked_widget = QStackedWidget()

        self.language_selection_page = LanguageSelectionPage(self.stacked_widget)
        self.morph_page = MorphSplitPage(self.stacked_widget)

        self.stacked_widget.addWidget(self.language_selection_page)
        self.stacked_widget.addWidget(self.morph_page)

        layout = QVBoxLayout()
        layout.addWidget(self.stacked_widget)
        self.setLayout(layout)

        self.stacked_widget.setCurrentIndex(0)

def main():
    app = QApplication(sys.argv)

    # Bigger global font
    font = QtGui.QFont()
    font.setPointSize(20)
    app.setFont(font)

    window = MainWindow()
    window.resize(900, 600)
    window.show()

    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


[DEBUG] Successfully loaded data: {'Russian': ['отличается', 'находит']}


2025-02-07 14:19:26.050 python[13082:590850] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-07 14:19:26.050 python[13082:590850] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [2]:
questions = {
  "questions": [
    {
      "question": "哪一個俄語單詞表示「你好」？",
      "options": {
        "A": "Спасибо",
        "B": "Привет",
        "C": "До свидания",
        "D": "Извините"
      },
      "correct_answer": "B",
      "explanation": "✅ **B) Привет** 是俄語中非正式的「你好」。\n\n- **Спасибо**（謝謝）\n- **До свидания**（再見）\n- **Извините**（對不起/打擾了）"
    },
    {
      "question": "俄語單詞 **\"Спасибо\"** 的意思是什麼？",
      "options": {
        "A": "請",
        "B": "對不起",
        "C": "謝謝",
        "D": "再見"
      },
      "correct_answer": "C",
      "explanation": "✅ **C) \"Спасибо\"** 的意思是 **\"謝謝\"**。\n\n其他選項：\n- **A) Пожалуйста**（請/不客氣）\n- **B) Извините**（對不起）\n- **D) До свидания**（再見）"
    },
    {
      "question": "在俄語中，如何說「請」這個詞？",
      "options": {
        "A": "Привет",
        "B": "Пожалуйста",
        "C": "До свидания",
        "D": "Извините"
      },
      "correct_answer": "B",
      "explanation": "✅ **B) Пожалуйста** 的意思是 **\"請\" 或 \"不客氣\"**。\n\n其他選項：\n- **Привет**（你好）\n- **До свидания**（再見）\n- **Извините**（對不起）"
    },
    {
      "question": "哪個詞表示「水」？",
      "options": {
        "A": "Еда",
        "B": "Вода",
        "C": "Привет",
        "D": "Извините"
      },
      "correct_answer": "B",
      "explanation": "✅ **B) Вода** 是俄語中的 \"水\"。\n\n其他選項：\n- **Еда**（食物）\n- **Привет**（你好）\n- **Извините**（對不起）"
    },
    {
      "question": "俄語中的 \"Еда\" 代表什麼？",
      "options": {
        "A": "水",
        "B": "食物",
        "C": "請",
        "D": "對不起"
      },
      "correct_answer": "B",
      "explanation": "✅ **B) Еда** 表示 **\"食物\"**。\n\n其他選項：\n- **A) Вода**（水）\n- **C) Пожалуйста**（請）\n- **D) Извините**（對不起）"
    },
    {
      "question": "哪一個選項是俄語的「再見」？",
      "options": {
        "A": "Спасибо",
        "B": "Извините",
        "C": "До свидания",
        "D": "Где"
      },
      "correct_answer": "C",
      "explanation": "✅ **C) До свидания** 表示 **\"再見\"**。\n\n其他選項：\n- **Спасибо**（謝謝）\n- **Извините**（對不起）\n- **Где**（哪裡）"
    },
    {
      "question": "當你想問「在哪裡？」時，你應該使用哪個俄語單詞？",
      "options": {
        "A": "Где",
        "B": "Нет",
        "C": "Да",
        "D": "Еда"
      },
      "correct_answer": "A",
      "explanation": "✅ **A) Где**（哪裡？）。\n\n其他選項：\n- **Нет**（不）\n- **Да**（是）\n- **Еда**（食物）"
    },
    {
      "question": "哪個選項是俄語的 \"是\"？",
      "options": {
        "A": "Нет",
        "B": "Да",
        "C": "Где",
        "D": "Спасибо"
      },
      "correct_answer": "B",
      "explanation": "✅ **B) Да**（是/對）。\n\n其他選項：\n- **Нет**（不）\n- **Где**（哪裡？）\n- **Спасибо**（謝謝）"
    },
    {
      "question": "哪個詞表示「不」？",
      "options": {
        "A": "Да",
        "B": "Где",
        "C": "Нет",
        "D": "Спасибо"
      },
      "correct_answer": "C",
      "explanation": "✅ **C) Нет**（不）。\n\n其他選項：\n- **Да**（是）\n- **Где**（哪裡？）\n- **Спасибо**（謝謝）"
    },
    {
      "question": "當你想道歉時，你應該說哪個詞？",
      "options": {
        "A": "Спасибо",
        "B": "Привет",
        "C": "Извините",
        "D": "Пожалуйста"
      },
      "correct_answer": "C",
      "explanation": "✅ **C) Извините**（對不起）。\n\n其他選項：\n- **Спасибо**（謝謝）\n- **Привет**（你好）\n- **Пожалуйста**（請/不客氣）"
    }
  ]
}


In [ ]:
import sys
import os
import json
import random  # <--- For shuffling
from PyQt5 import QtGui
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QLabel, QRadioButton,
    QPushButton, QButtonGroup, QMessageBox, QComboBox
)
from PyQt5.QtCore import Qt

JSON_FILE = "questions.json"


def load_data():
    """Load the entire JSON data which contains categories."""
    if not os.path.exists(JSON_FILE):
        print(f"Error: JSON file '{JSON_FILE}' not found.")
        return {}
    try:
        with open(JSON_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return data
    except json.JSONDecodeError as e:
        print("JSONDecodeError:", e)
        return {}


def get_categories(data):
    """Return the dictionary of categories from the loaded JSON."""
    return data.get("categories", {})


class CategorySelectionWindow(QWidget):
    """
    The 'main page', where user selects a category or exits.
    Also contains a method to rewrite the JSON file when questions are deleted.
    """
    def __init__(self, data):
        super().__init__()
        self.setWindowTitle("Select a Category")
        self.setStyleSheet("background-color: black;")

        # We'll keep the entire data in memory, so we can rewrite to JSON upon deletion
        self.data = data  # entire JSON content
        self.categories = get_categories(data)  # shortcut to data["categories"]

        # Make the main window bigger
        self.resize(1000, 600)

        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout(self)
        self.setLayout(layout)

        label = QLabel("Please select a language/category:")
        label.setStyleSheet("font-size: 18px; color: white; font-weight: bold;")
        layout.addWidget(label)

        self.combo_box = QComboBox()
        # White background for combo text
        self.combo_box.setStyleSheet("font-size: 16px; color: black; background-color: white;")
        layout.addWidget(self.combo_box)

        # Populate combo box with category names
        for cat_name in self.categories.keys():
            self.combo_box.addItem(cat_name)

        # Start quiz button
        start_button = QPushButton("Start Quiz")
        start_button.setStyleSheet("font-size: 16px; color: white; background-color: #333;")
        start_button.clicked.connect(self.start_quiz)
        layout.addWidget(start_button)

        # Exit program button
        exit_button = QPushButton("Exit Program")
        exit_button.setStyleSheet("font-size: 16px; color: white; background-color: #333;")
        exit_button.clicked.connect(self.exit_program)
        layout.addWidget(exit_button)

    def start_quiz(self):
        """Start the quiz with the selected category."""
        selected_category = self.combo_box.currentText()
        if not selected_category:
            QMessageBox.warning(self, "No Category", "Please select a category.")
            return

        questions = self.categories.get(selected_category, [])
        if not questions:
            QMessageBox.warning(
                self,
                "No Questions",
                f"No questions found for category {selected_category}."
            )
            return

        # *** Shuffle the questions so they come up in random order ***
        random.shuffle(questions)

        # Create the quiz window, passing a reference to self (the main window)
        self.quiz_window = QuizWindow(questions, selected_category, self)
        self.quiz_window.show()

        # Hide the main window (instead of closing) so we can come back later
        self.hide()

    def exit_program(self):
        """Completely quit the application."""
        QApplication.instance().quit()

    def write_json_to_file(self):
        """
        Rewrite the entire data (self.data) to the JSON file.
        This is called when a question is deleted to make the deletion permanent.
        """
        with open(JSON_FILE, "w", encoding="utf-8") as f:
            json.dump(self.data, f, indent=4, ensure_ascii=False)


class QuizWindow(QWidget):
    """
    The quiz window for a chosen category.
    Receives:
      - questions (list of question dicts)
      - category_name (string)
      - main_window (CategorySelectionWindow reference) to access data, show/hide windows, etc.
    """
    def __init__(self, questions, category_name, main_window):
        super().__init__()
        self.setWindowTitle(f"Vocabulary Quiz - {category_name}")
        self.setStyleSheet("background-color: black;")

        # Make this window bigger as well
        self.resize(1000, 600)

        self.questions = questions
        self.current_question_index = 0
        self.category_name = category_name
        self.main_window = main_window  # Reference to CategorySelectionWindow

        self.init_ui()

        if self.questions:
            self.show_question(0)
        else:
            QMessageBox.warning(self, "No Questions", "No questions found for this category.")
            self.disable_quiz_ui()

    def init_ui(self):
        layout = QVBoxLayout(self)
        self.setLayout(layout)

        self.question_label = QLabel("Question will appear here.")
        self.question_label.setStyleSheet("font-size: 18px; color: white; font-weight: bold;")
        layout.addWidget(self.question_label)

        self.button_group = QButtonGroup(self)
        self.radio_buttons = []
        for _ in range(4):  # up to 4 options
            rb = QRadioButton()
            rb.setStyleSheet("font-size: 16px; color: white;")
            self.radio_buttons.append(rb)
            self.button_group.addButton(rb)
            layout.addWidget(rb)

        self.next_button = QPushButton("Next Question")
        self.next_button.setStyleSheet("font-size: 16px; color: white; background-color: #333;")
        self.next_button.clicked.connect(self.check_and_next)
        layout.addWidget(self.next_button)

        # A "Delete" button to remove the current question from JSON
        self.delete_button = QPushButton("Delete This Question")
        self.delete_button.setStyleSheet("font-size: 16px; color: white; background-color: #333;")
        self.delete_button.clicked.connect(self.delete_question)
        layout.addWidget(self.delete_button)

        # A "Back to Main" button to leave the quiz early
        self.back_button = QPushButton("Back to Main")
        self.back_button.setStyleSheet("font-size: 16px; color: white; background-color: #333;")
        self.back_button.clicked.connect(self.back_to_main)
        layout.addWidget(self.back_button)

    def show_question(self, index):
        """Load question text + options into the UI."""
        q_data = self.questions[index]
        question_text = q_data.get("question", "")
        options_dict = q_data.get("options", {})

        self.question_label.setText(question_text)

        # Temporarily disable exclusivity to reset selection
        self.button_group.setExclusive(False)
        for rb in self.radio_buttons:
            rb.setChecked(False)
            rb.setStyleSheet("font-size: 16px; color: white;")
            rb.hide()
        self.button_group.setExclusive(True)

        # Sort keys (A, B, C, D) for consistent order
        sorted_keys = sorted(options_dict.keys())
        for i, key in enumerate(sorted_keys):
            if i < len(self.radio_buttons):
                text = f"{key}) {options_dict[key]}"
                self.radio_buttons[i].setText(text)
                self.radio_buttons[i].show()

        # If fewer than 4 options, hide the extra radio buttons
        for j in range(len(sorted_keys), len(self.radio_buttons)):
            self.radio_buttons[j].hide()

    def check_and_next(self):
        """When user clicks 'Next Question': check the answer, show popup, then load next."""
        if not self.questions:
            return

        selected_rb = self.button_group.checkedButton()
        if not selected_rb:
            QMessageBox.warning(self, "No Selection", "Please select an option.")
            return

        selected_text = selected_rb.text()  # e.g. "B) Привет"
        selected_letter = selected_text.split(")")[0].strip()

        q_data = self.questions[self.current_question_index]
        correct_letter = q_data.get("correct_answer", "").strip()
        explanation = q_data.get("explanation", "")

        # Remove all '**' (Markdown bold) from explanation
        explanation_no_asterisks = explanation.replace("**", "")

        # Check correctness
        if selected_letter == correct_letter:
            selected_rb.setStyleSheet("font-size: 16px; color: green; font-weight: bold;")
            QMessageBox.information(
                self, "Correct",
                explanation_no_asterisks or "Good job!"
            )
        else:
            selected_rb.setStyleSheet("font-size: 16px; color: red; font-weight: bold;")
            # highlight correct
            for rb in self.radio_buttons:
                if rb.text().startswith(correct_letter + ")"):
                    rb.setStyleSheet("font-size: 16px; color: green; font-weight: bold;")
                    break
            QMessageBox.warning(
                self, "Incorrect",
                f"正確答案是：{correct_letter}\n\n{explanation_no_asterisks}"
            )

        # Move to next question
        self.current_question_index += 1
        if self.current_question_index < len(self.questions):
            self.show_question(self.current_question_index)
        else:
            QMessageBox.information(self, "Quiz Finished", "You have reached the end of the quiz!")
            self.back_to_main()

    def delete_question(self):
        """
        Delete the current question from the in-memory list AND from the JSON file.
        Then proceed to next question or back to main if no questions remain.
        """
        if not self.questions:
            return

        # Confirm user wants to delete
        reply = QMessageBox.question(
            self, "Delete Question?",
            "Are you sure you want to PERMANENTLY delete this question from the JSON file?",
            QMessageBox.Yes | QMessageBox.No
        )
        if reply == QMessageBox.No:
            return  # do nothing

        # Remove the current question from the list
        self.questions.pop(self.current_question_index)

        # Also remove it from the main_window.categories for this category
        self.main_window.categories[self.category_name] = self.questions

        # Overwrite JSON on disk so the deletion is permanent
        self.main_window.data["categories"] = self.main_window.categories
        self.main_window.write_json_to_file()

        # If after removing the question there's no more question at the same index,
        # we might either go back or see if there's a next question to display
        if self.current_question_index >= len(self.questions):
            # No more questions in this category
            QMessageBox.information(self, "Deleted", "Question deleted. No more questions remain.")
            self.back_to_main()
        else:
            # Show the next question at the same index (since we popped the old one)
            self.show_question(self.current_question_index)

    def back_to_main(self):
        """
        Return to the main page (CategorySelectionWindow).
        Closes the QuizWindow and re-shows the main window.
        """
        if self.main_window is not None:
            self.main_window.show()
        self.close()

    def disable_quiz_ui(self):
        """Disable the quiz UI if needed (e.g., no questions from the start)."""
        self.question_label.setText("No more questions.")
        for rb in self.radio_buttons:
            rb.hide()
        self.next_button.setEnabled(False)
        self.delete_button.setEnabled(False)


def main():
    app = QApplication(sys.argv)

    # Increase default font size so all text is bigger
    font = QtGui.QFont()
    font.setPointSize(20)
    app.setFont(font)

    # Load JSON data (entire structure)
    data = load_data()
    categories = get_categories(data)

    if not categories:
        msg = "No categories found in the JSON file."
        print(msg)
        error_win = QWidget()
        error_win.setWindowTitle("Error")
        layout = QVBoxLayout(error_win)
        lbl = QLabel(msg)
        layout.addWidget(lbl)
        error_win.show()
        sys.exit(app.exec_())
    else:
        # Show the category selection window
        window = CategorySelectionWindow(data)
        # window.resize(...) is called inside CategorySelectionWindow.__init__
        window.show()
        sys.exit(app.exec_())


if __name__ == "__main__":
    main()


SystemExit: 0

: 